In [5]:
%load_ext iminizinc
%load_ext autoreload
%autoreload 2

from __future__ import print_function
from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual, HBox, VBox, Output
from ipywidgets import IntSlider, IntText, FloatSlider, FloatText, IntRangeSlider, FloatRangeSlider
# import ipywidgets as widgets

from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6


import sys, os


# import mplfinance as mpf
%matplotlib inline

# plt.rcParams['lines.linewidth'] = 0.5 



The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
apr = 0.06

periods_per_year = 12

periodic_rate = apr/periods_per_year

apy = (1 + periodic_rate) ** periods_per_year - 1
apy

0.06167781186449828

### APR to APY

In [40]:

aprs = np.arange(0,0.1,0.001)
periods_per_year = 365
periodic_rate = apr/periods_per_year
apys = np.power( (1 + aprs/periods_per_year) , periods_per_year) - 1
apys = ((1 + aprs/periods_per_year) ** periods_per_year - 1).round(4)
df_apr_apy = pd.DataFrame([aprs,apys]).T.set_axis(['apr', 'apy'], axis=1).set_index('apr')

@interact(sapr=(0,0.1,0.001))
def print_apr_apy(sapr=0.06):
    fig = plt.figure(figsize=(15,5))
    ax = fig.gca()
    ax.set_xlabel('APR')
    ax.set_ylabel('APY')
    ax.set_title('APR vs API')

    sapy =  (1 + sapr/periods_per_year) ** periods_per_year - 1
    ax.axvline(sapr)
    ax.axhline(sapy)
    print(f'APR = {sapr} -> APY = {sapy}')
    # ax.set_xscale('log')
    plt.plot(aprs, apys)
    ax.grid()
    # (1 + periodic_rate)

NameError: name 'apr' is not defined

### Amortizing Loan

In [97]:

aprs = np.arange(0.001,0.1,0.001)
periods_per_year = 12

wdg_P = IntText(value=100000,min=0,max=3000000,step=10000,description='P',continuous_update=False)
# wdg_P = IntSlider(value=500000,min=0,max=1000000,step=10000,description='P',continuous_update=False)
wdg_n_years = IntSlider(value=20,min=1,max=35,step=1,description='nyears',continuous_update=False)
wdg_check_apr = FloatText(value=0.055,min=0,max=0.1,step=0.001,description='check',continuous_update=False)

ui = VBox([
    HBox([wdg_P, wdg_n_years, wdg_check_apr]),
    Output()
])

def print_amortizing_periodic_payments(P, n_years, check_apr):
    n_periods = periods_per_year * n_years
    r_s = aprs / periods_per_year
    r1n_s = (1 + r_s) ** n_periods
    
    Q_s = r_s * r1n_s / (r1n_s - 1)
    A_s = P * Q_s
    T_s = A_s * n_periods
    # period_A_s = A_s / n_periods

    check_r = check_apr / periods_per_year
    check_r1n = (1 + check_r) ** n_periods
    check_A = round(P * check_r * check_r1n / (check_r1n - 1) , 2)
    check_total = round(check_A * n_periods, 2)
    check_interest_2_principal = round(check_total / P - 1, 2)
    print(f'Periodic payment: {check_A}, Total payment: {check_total}, interest_2_principal: {check_interest_2_principal}')

    plt.ioff()
    plt.close('all')
    fig = plt.figure(figsize=(15,7))
    ax1, ax2 = fig.subplots(2,1, sharex=True)
    plt.tight_layout(h_pad=3)

    ax2.plot(aprs, T_s)
    ax2.set_xlabel('APR')
    ax2.set_ylabel('Total repayment')
    ax2.set_title('APR vs Total repayment')

    ax1.plot(aprs, A_s)
    ax1.set_xlabel('APR')
    ax1.set_ylabel('Monthly repayment')
    ax1.set_title('APR vs Monthly repayment')

    for ax in [ax1,ax2]:
        ax.grid()
        ax.axvline(check_apr)

    ax1.axhline(check_A)

    plt.show()



out = interactive_output(print_amortizing_periodic_payments, {
        'P': wdg_P, 'n_years': wdg_n_years, 'check_apr': wdg_check_apr
    });
x = display(ui, out);


Output()

### Repayments over different mortgage periods

In [89]:

aprs = np.arange(0.001,0.1,0.001).round(3)
periods_per_year = 12

rng_n_years, step_n_years = (5,30), 5
n_years_s = [3, 5, 7, 10, 15, 20, 25, 30]

wdg_P = IntText(value=100000,min=0,max=3000000,step=10000,description='P',continuous_update=False)
wdg_check_apr = FloatText(value=0.055,min=0,max=0.1,step=0.001,description='check',continuous_update=False)

ui = VBox([
    HBox([wdg_P, wdg_check_apr]),
    Output()
])

def print_amortizing_periodic_payments(P, check_apr):
    df = pd.concat({
        f'{n_years}-years': pd.DataFrame([A_s, T_s, T_Int, R_T2P]).T.set_axis(['Monthly', 'Total', 'Total Interest', 'Interest2Principal'],axis=1)
        for n_years in n_years_s
        for n_periods in [periods_per_year * n_years]
        for r_s in [aprs / periods_per_year]
        for r1n_s in [(1 + r_s) ** n_periods]
        for Q_s in [r_s * r1n_s / (r1n_s - 1)]
        for A_s in [P * Q_s]
        for T_s in [A_s * n_periods]
        for T_Int in [T_s - P]
        for R_T2P in [T_s / P -1]
    },axis=1).set_index(aprs)
    df_monthly = df.loc[:, (slice(None),'Monthly')]
    df_total = df.loc[:, (slice(None),'Total')]
    print(df.loc[check_apr].unstack().round(2))

    plt.ioff()
    plt.close('all')
    fig = plt.figure(figsize=(15,7))
    ax1, ax2 = fig.subplots(2,1, sharex=True)
    plt.tight_layout(h_pad=3)

    df_monthly.plot(ax=ax1)
    ax1.set_xlabel('APR')
    ax1.set_ylabel('Monthly repayment')
    ax1.set_title('APR vs Monthly repayment')

    df_total.plot(ax=ax2)
    ax2.set_xlabel('APR')
    ax2.set_ylabel('Total repayment')
    ax2.set_title('APR vs Total repayment')

    for ax in [ax1,ax2]:
        ax.grid()
        ax.axvline(check_apr)

    plt.show()

out = interactive_output(print_amortizing_periodic_payments, {
        'P': wdg_P, 'check_apr': wdg_check_apr
    });
x = display(ui, out);


Output()

### Monthy Repayments Interest vs Principal 

In [105]:

aprs = np.arange(0.001,0.1,0.001).round(3)
periods_per_year = 12

rng_n_years, step_n_years = (5,30), 5
n_years_s = list(range(rng_n_years[0], rng_n_years[1] + step_n_years, step_n_years))
n_years_s = [3, 5, 7, 10, 15, 20, 25, 30]

wdg_P = IntText(value=300000,min=0,max=3000000,step=10000,description='P',continuous_update=False)
wdg_n_years = IntSlider(value=20,min=1,max=35,step=1,description='nyears',continuous_update=False)
wdg_check_apr = FloatText(value=0.055,min=0,max=0.1,step=0.001,description='check',continuous_update=False)

ui = VBox([
    HBox([wdg_P, wdg_check_apr, wdg_n_years]),
    Output()
])

def print_monthly_interst_vs_principal(P, n_years, apr):

    # df = pd.concat({
    #     f'{n_years}-years': pd.DataFrame([A_s, T_s, T_Int, R_T2P]).T.set_axis(['Monthly', 'Total', 'Total Interest', 'Interest2Principal'],axis=1)
    #     for n_years in n_years_s
    #     for n_periods in [periods_per_year * n_years]
    #     for r_s in [aprs / periods_per_year]
    #     for r1n_s in [(1 + r_s) ** n_periods]
    #     for Q_s in [r_s * r1n_s / (r1n_s - 1)]
    #     for A_s in [P * Q_s]
    #     for T_s in [A_s * n_periods]
    #     for T_Int in [T_s - P]
    #     for R_T2P in [T_s / P -1]
    # },axis=1).set_index(aprs)
    # df_monthly = df.loc[:, (slice(None),'Monthly')]
    # df_total = df.loc[:, (slice(None),'Total')]
    # print(df.loc[check_apr].unstack().round(2))


    n_periods = periods_per_year * n_years
    r_s = apr / periods_per_year
    tot_1n_s = (1 + r_s) ** n_periods
    
    Q_s = r_s * tot_1n_s / (tot_1n_s - 1)
    A_s = P * Q_s
    T_s = A_s * n_periods    

    a_periods = np.array(range(n_periods))
    a_remaining = n_periods - a_periods

    rem_1n_s = (1 + r_s) ** a_remaining

    a_outstanding_balance = P * ((1 + r_s) ** a_remaining - tot_1n_s) / (rem_1n_s - 1)
    a_interest = a_outstanding_balance * r_s
    a_principal = A_s - a_interest
    
    df = pd.DataFrame([a_outstanding_balance, a_interest, a_principal]).T.set_axis(['outstanding_balance', 'interest', 'principal'], axis=1)
    # Outstanding Loan Balance = (Loan Amount x ((1 + Monthly Interest Rate)^Remaining Payments - (1 + Monthly Interest Rate)^Total Number of Months)) / ((1 + Monthly Interest Rate)^Remaining Payments - 1)

    plt.ioff()
    plt.close('all')
    fig = plt.figure(figsize=(15,7))
    ax1, ax2 = fig.subplots(2,1, sharex=True)
    plt.tight_layout(h_pad=3)
    ax1.plot(a_outstanding_balance)
    # df['outstanding_balance'].plot(ax=ax1)
    # df_monthly.plot(ax=ax1)
    # ax1.set_xlabel('APR')
    # ax1.set_ylabel('Monthly repayment')
    # ax1.set_title('APR vs Monthly repayment')

    # df_total.plot(ax=ax2)
    # ax2.set_xlabel('APR')
    # ax2.set_ylabel('Total repayment')
    # ax2.set_title('APR vs Total repayment')

    # for ax in [ax1,ax2]:
    #     ax.grid()
    #     ax.axvline(check_apr)

    plt.show()

out = interactive_output(print_monthly_interst_vs_principal, {
        'P': wdg_P, 'apr': wdg_check_apr, 'n_years': wdg_n_years
    });
x = display(ui, out);


Output()

In [115]:
n_years = 20
P = 300000
apr = 0.06
periods_per_year = 12

n_periods = periods_per_year * n_years

rM = apr/periods_per_year

# A is the monthly payment amount. A is derived from the above paremeters
A = P * (rM*np.power((1+rM), n_periods))/(np.power((1+rM),n_periods)-1)
A
# r_s = apr / periods_per_year
# tot_1n_s = (1 + r_s) ** n_periods

# Q_s = r_s * tot_1n_s / (tot_1n_s - 1)
# A_s = P * Q_s
# T_s = A_s * n_periods    

# a_periods = np.array(range(n_periods))
# a_remaining = n_periods - a_periods

# rem_1n_s = (1 + r_s) ** a_remaining
# # rem_1n_s
# a_outstanding_balance = P * ((1 + r_s) ** a_remaining - tot_1n_s) / (rem_1n_s - 1)
# a_outstanding_balance
# a_interest = a_outstanding_balance * r_s
# a_principal = A_s - a_interest


2149.2931754345186